In [36]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Dense, BatchNormalization, Dropout, Flatten, Activation
from keras.layers.convolutional import ZeroPadding2D
from keras.models import Sequential
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.layers.core import Lambda
from keras import initializers
from keras.utils import get_file
import tensorflow as tf
import numpy as np
import os

In [37]:
import collections

path = '/work/04381/ymarathe/maverick/yearbook/keras_yearbook/'
data_path = '/work/04381/ymarathe/maverick/yearbook/'

f = open(data_path + 'yearbook_train.txt', 'r')

freq = {};
normal_const = 0;

for line in f:
    line = line.rstrip()
    image, year = line.split("\t")
    if year in freq:
        freq[year] += 1
    else:
        freq[year] = 1

normal_const = np.sum(freq.values())
for key in freq:
    freq[key] = freq[key]/float(normal_const);
    
sorted_freq = collections.OrderedDict(sorted(freq.items()))

idx = 0;
class_weights_train = {}
idx2year = {}

for key in sorted_freq:
    class_weights_train[idx] = sorted_freq[key]
    idx2year[idx] = key
    idx += 1

In [38]:
def gen_batches(path, gen = ImageDataGenerator(), shuffle=True, class_mode="categorical", batch_size=32, 
                target_size=(171, 186)):
    return gen.flow_from_directory(path, shuffle=shuffle, batch_size=batch_size, target_size=target_size, 
                                   class_mode=class_mode)

def gen_batches_flow(path, gen = ImageDataGenerator(), shuffle=True, batch_size=32):
    return gen.flow(path, shuffle=shuffle, batch_size=batch_size)

In [39]:
#female_train = gen_batches(path + 'train/F')
#male_train = gen_batches(path + 'train/M')
#female_valid = gen_batches(path + 'valid/F')
#male_valid = gen_batches(path + 'valid/M')

In [40]:
#train = gen_batches(path + 'train')
#valid = gen_batches(path + 'valid', shuffle=False)

In [41]:
def vgg19_conv_layers_sequential():
    model = Sequential()
    
    model.add(ZeroPadding2D((1, 1), input_shape=(171, 186, 3)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))

    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    
    vgg_pretrain_weights = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    "https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5",
                                    cache_subdir='models')
    
    model.load_weights(vgg_pretrain_weights)
    
    return model

In [42]:
#with tf.device('/gpu:0'):
        #model.fit_generator(female_train, steps_per_epoch = female_train.samples/female_train.batch_size, epochs=1,
                    #validation_data=female_valid, validation_steps = female_valid.samples/female_valid.batch_size)
        #model.fit_generator(male_train, steps_per_epoch = male_train.samples/male_train.batch_size, epochs=1,
                    #validation_data=male_valid, validation_steps = male_valid.samples/male_valid.batch_size)
        #model.fit_generator(train, steps_per_epoch = train.samples/train.batch_size, epochs = 5, 
                            #validation_data = valid, 
                            #validation_steps = valid.samples/valid.batch_size)
        #model.save_weights(path + 'weights_exp6.h5')

**Experiment 1**
- All layers trained
- lr = 1e-4
- Adam
- Dense layers weight initialized 
- OVERFITTING is apparent
- Epochs = 1

Total params: 89,664,680

Trainable params: 89,664,680

Non-trainable params: 0

Epoch 1/1

379/379 [==============================] - 668s - loss: 5.1517 - acc: 0.0237 - val_loss: 4.7387 - val_acc: 0.0223

Epoch 1/1

334/334 [==============================] - 578s - loss: 4.1187 - acc: 0.0723 - val_loss: 4.4772 - val_acc: 0.0245

**Experiment 2**
- Motivation: loss function keeps decreasing in Experiment 1, maybe more epochs will increase accuracy?
- Dense layers trained
- lr = 1e-5
- Dense layers weight initialized 
- alternatively improving values
- Overfitting or not is not clear
- Epochs = 5

=================================================================

Total params: 89,664,680

Trainable params: 69,640,296

Non-trainable params: 20,024,384

_________________________________________________________________

Epoch 1/1

379/379 [==============================] - 240s - loss: 13.6884 - acc: 0.0358 - val_loss: 11.4914 - val_acc: 0.0251

Epoch 1/1

334/334 [==============================] - 203s - loss: 11.5692 - acc: 0.0666 - val_loss: 9.2627 - val_acc: 0.0127

Epoch 1/1

379/379 [==============================] - 237s - loss: 8.4670 - acc: 0.1118 - val_loss: 6.3364 - val_acc: 0.0365

Epoch 1/1

334/334 [==============================] - 204s - loss: 6.4200 - acc: 0.1361 - val_loss: 5.8004 - val_acc: 0.0133

Epoch 1/1

379/379 [==============================] - 234s - loss: 5.2417 - acc: 0.1739 - val_loss: 5.1995 - val_acc: 0.0289

Epoch 1/1

334/334 [==============================] - 203s - loss: 4.6242 - acc: 0.1942 - val_loss: 5.1862 - val_acc: 0.0220

Epoch 1/1

379/379 [==============================] - 234s - loss: 4.0165 - acc: 0.2328 - val_loss: 4.8735 - val_acc: 0.0351

Epoch 1/1

334/334 [==============================] - 202s - loss: 3.7839 - acc: 0.2412 - val_loss: 5.0708 - val_acc: 0.0202

Epoch 1/1

379/379 [==============================] - 234s - loss: 3.4204 - acc: 0.2830 - val_loss: 4.7858 - val_acc: 0.0347

Epoch 1/1

334/334 [==============================] - 202s - loss: 3.2999 - acc: 0.2905 - val_loss: 5.0043 - val_acc: 0.0202


**Experiment 3**
- Will batchnormalization help since theres is no improvement in validation accuracy
- Weight initialized dense layers
- Only dense layers trainable
- Batchnormalized dense layers
- Dropout = 0.5
- lr = 1e-4
- Overfits

=================================================================

Total params: 89,697,448

Trainable params: 69,640,296

Non-trainable params: 20,057,152

_________________________________________________________________

Epoch 1/1

379/379 [==============================] - 242s - loss: 5.0252 - acc: 0.0388 - val_loss: 4.4513 - val_acc: 0.0190

Epoch 1/1

334/334 [==============================] - 205s - loss: 4.5141 - acc: 0.0941 - val_loss: 4.6469 - val_acc: 0.0136


**Experiment 4**
- Motivation: Overfitting in Experiment 3, do more dropout
- Weight initialized dense layers
- Only dense layers trainable
- Batchnormalization with dense layers
- Dropout = 0.7
- lr = 1e-4
- Not much improvement

Total params: 89,697,448

Trainable params: 69,640,296

Non-trainable params: 20,057,152

Epoch 1/1

379/379 [==============================] - 240s - loss: 5.4357 - acc: 0.0119 - val_loss: 4.5188 - val_acc: 0.0255

Epoch 1/1

334/334 [==============================] - 204s - loss: 5.2047 - acc: 0.0261 - val_loss: 4.5505 - val_acc: 0.0127



**Experiment 5**
- Motivation: Loss keeps decreasing in Experiment 4, do more number of epochs
- Weight initialized dense layers
- Only dense layers trainable
- Batchnormalization with dense layers
- Dropout = 0.7
- lr = 1e-4
- Validation loss is not stable across epochs


Epoch 1/1

379/379 [==============================] - 240s - loss: 5.4000 - acc: 0.0125 - val_loss: 4.4933 - val_acc: 0.0381

Epoch 1/1

334/334 [==============================] - 205s - loss: 5.2335 - acc: 0.0199 - val_loss: 4.5120 - val_acc: 0.0213

Epoch 1/1

379/379 [==============================] - 239s - loss: 4.8773 - acc: 0.0416 - val_loss: 4.4056 - val_acc: 0.0206

Epoch 1/1

334/334 [==============================] - 205s - loss: 4.7485 - acc: 0.0563 - val_loss: 4.4966 - val_acc: 0.0138

Epoch 1/1

379/379 [==============================] - 236s - loss: 4.4608 - acc: 0.0814 - val_loss: 4.3931 - val_acc: 0.0184

Epoch 1/1

334/334 [==============================] - 204s - loss: 4.3454 - acc: 0.1029 - val_loss: 4.5639 - val_acc: 0.0119

Epoch 1/1

379/379 [==============================] - 236s - loss: 4.1044 - acc: 0.1265 - val_loss: 4.4471 - val_acc: 0.0217

Epoch 1/1

334/334 [==============================] - 204s - loss: 4.0820 - acc: 0.1358 - val_loss: 4.6092 - val_acc: 0.0133

Epoch 1/1

379/379 [==============================] - 236s - loss: 3.8478 - acc: 0.1620 - val_loss: 4.4567 - val_acc: 0.0231

Epoch 1/1

334/334 [==============================] - 204s - loss: 3.9038 - acc: 0.1616 - val_loss: 4.6779 - val_acc: 0.0119


**Experiment 6**
- No separation of male and female data
- lr = 1e-4
- 3 epochs
- BatchNorm with dense layers
- Dense layers weight initialized
- Only Dense layers trained
- Training accuracy is very very high compared to accuracy on validation set
- No improvement in validation set accuracy

Total params: 89,697,448

Trainable params: 69,640,296

Non-trainable params: 20,057,152

Epoch 1/5

708/708 [==============================] - 443s - loss: 3.5136 - acc: 0.2312 - val_loss: 5.0918 - val_acc: 0.0282

Epoch 2/5

708/708 [==============================] - 441s - loss: 2.3899 - acc: 0.4080 - val_loss: 5.2752 - val_acc: 0.0249

Epoch 3/5

708/708 [==============================] - 437s - loss: 1.9634 - acc: 0.4918 - val_loss: 5.4608 - val_acc: 0.0282

Epoch 4/5

708/708 [==============================] - 436s - loss: 1.6964 - acc: 0.5481 - val_loss: 5.6526 - val_acc: 0.0278



**Experiment 7**
- No separation of male and female data
- Use only one dense layer
- No dropout
- Everything else stays same as Experiment 6
- WOW! training accuracy is 96.9% after 3 epochs
- WOW^2! training accuracy is 99.3% after 4 epochs
- BUT... validation accuracy is abysmal! Validation loss keeps increasing!
- QUESTION: WHY does the network NOT generalize well?
- Is the training set so different from validation set?

Epoch 1/4
 
708/708 [==============================] - 426s - loss: 2.3755 - acc: 0.4281 - val_loss: 5.6640 - val_acc: 0.0308

Epoch 2/4

708/708 [==============================] - 426s - loss: 0.6264 - acc: 0.8555 - val_loss: 6.1181 - val_acc: 0.0268

Epoch 3/4

708/708 [==============================] - 422s - loss: 0.2287 - acc: 0.9644 - val_loss: 6.2639 - val_acc: 0.0211

Epoch 4/4

708/708 [==============================] - 422s - loss: 0.0844 - acc: 0.9935 - val_loss: 6.3802 - val_acc: 0.0231


**Experiment 8**
- No separation of male and female data
- Use only one dense layer
- 50% dropout
- Everything else stays same as Experiment 6
- Even dropout doesn't help

Total params: 72,899,752

Trainable params: 52,858,984

Non-trainable params: 20,040,768

_________________________________________________________________
Epoch 1/4
708/708 [==============================] - 427s - loss: 2.8264 - acc: 0.3509 - val_loss: 5.4841 - val_acc: 0.0229

Epoch 2/4

708/708 [==============================] - 427s - loss: 1.1657 - acc: 0.6901 - val_loss: 5.6196 - val_acc: 0.0297

Epoch 3/4

708/708 [==============================] - 422s - loss: 0.6635 - acc: 0.8322 - val_loss: 5.8154 - val_acc: 0.0342

Epoch 4/4

708/708 [==============================] - 422s - loss: 0.3946 - acc: 0.9108 - val_loss: 6.2051 - val_acc: 0.0299


In [43]:
model = vgg19_conv_layers_sequential()
model.add(Flatten())
model.add(Dense(4096, kernel_initializer='glorot_normal', bias_initializer=keras.initializers.Ones()))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(104, activation='softmax'))

for layer in model.layers:
    if type(layer) != Dense:
        layer.trainable = False

model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 173, 188, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 171, 186, 64)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 173, 188, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 171, 186, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 85, 93, 64)        0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 87, 95, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 85, 93, 128)       73856     
__________

**Experiment 9**
- Two dense layers with class weights to scale the loss function
- Number of epochs = 10
- Nothing improvesTotal params: 72,899,752
Trainable params: 52,858,984
Non-trainable params: 20,040,768
_________________________________________________________________
Epoch 1/10

708/708 [==============================] - 423s - loss: 0.0387 - acc: 0.3163 - val_loss: 5.8928 - val_acc: 0.0223

Epoch 2/10

708/708 [==============================] - 423s - loss: 0.0162 - acc: 0.6341 - val_loss: 6.0068 - val_acc: 0.0301

Epoch 3/10

708/708 [==============================] - 419s - loss: 0.0094 - acc: 0.7735 - val_loss: 6.1302 - val_acc: 0.0258

Epoch 4/10

708/708 [==============================] - 419s - loss: 0.0060 - acc: 0.8587 - val_loss: 6.2693 - val_acc: 0.0337

Epoch 5/10

708/708 [==============================] - 419s - loss: 0.0040 - acc: 0.9068 - val_loss: 6.6697 - val_acc: 0.0211

Epoch 6/10

708/708 [==============================] - 418s - loss: 0.0027 - acc: 0.9402 - val_loss: 6.6209 - val_acc: 0.0313

Epoch 7/10

708/708 [==============================] - 419s - loss: 0.0019 - acc: 0.9605 - val_loss: 6.8412 - val_acc: 0.0229

Epoch 8/10

708/708 [==============================] - 419s - loss: 0.0015 - acc: 0.9699 - val_loss: 7.3279 - val_acc: 0.0207

Epoch 9/10

708/708 [==============================] - 419s - loss: 0.0014 - acc: 0.9744 - val_loss: 6.9323 - val_acc: 0.0317

Epoch 10/10

708/708 [==============================] - 419s - loss: 0.0011 - acc: 0.9785 - val_loss: 7.2606 - val_acc: 0.0270


In [44]:
import keras.backend as K
from keras.preprocessing.image import load_img, img_to_array

class PseudoLabeling:
    def __init__(self, model, train, valid, path = None):
        self.model = model
        self.train = train
        self.valid = valid
        self.path = path
        self.val_pred_labels = None
        
    def predict_labels(self, batch):
        pred_probs = self.model.predict_generator(batch, steps = ((batch.samples/batch.batch_size) + 1))
        pred_labels = np.zeros_like(pred_probs)
        pred_labels[np.arange(len(pred_labels)), pred_probs.argmax(1)] = 1
        return pred_labels
    
    def predict_validation_labels(self):
        self.val_pred_labels = self.predict_labels(self.valid)
        
    def reset_validation_labels(self):
        self.val_pred_labels = None
        
    def get_validation_labels(self, idx_array = None):
        if self.val_pred_labels is None:
            raise Exception('Validation labels not populated. \
                            Call predict_validation_labels on Pseudolabeling object \
                            to populate validation labels first')
        validation_labels = []
        if idx_array is None:
            return self.val_pred_labels
        else:
            for idx in idx_array:
                validation_labels.append(self.val_pred_labels[idx])
                
            return validation_labels   

    def next(self, *args, **kwargs):
        batch_train_x, batch_train_y = next(self.train)
        
        batch_valid_idx_array, batch_valid_cur_idx, batch_size = next(self.valid.index_generator)
        batch_valid_x = np.zeros((batch_size,) + self.valid.image_shape, dtype=K.floatx())
        grayscale = self.valid.color_mode == 'grayscale'
        
        for i, j in enumerate(batch_valid_idx_array):
            fname = self.valid.filenames[j]
            img = load_img(
                  os.path.join(self.valid.directory, fname),
                  grayscale=grayscale,
                  target_size=self.valid.target_size)
            x = img_to_array(img, data_format=self.valid.data_format)
            x = self.valid.image_data_generator.random_transform(x)
            x = self.valid.image_data_generator.standardize(x)
            batch_valid_x[i] = x
            
        #if self.val_pred_labels is None:
        #    batch_valid_y = np.zeros((len(batch_valid_x), self.valid.num_class), dtype=K.floatx())
        #    for i, label in enumerate(self.valid.classes[batch_valid_idx_array]):
        #        batch_valid_y[i, label] = 1.
        #else:
        batch_valid_y = self.get_validation_labels(batch_valid_idx_array);
        
        n0 = np.concatenate([batch_train_x, batch_valid_x])
        n1 = np.concatenate([batch_train_y, batch_valid_y])
        return (n0, n1)

In [45]:
#train = gen_batches(path + 'train', batch_size = 32)
#valid = gen_batches(path + 'valid', batch_size = 16, shuffle=False)

**Experiment 10**
- Pseudolabeling of the validation class
- Does not improve accuracy at all

In [46]:
#train_steps = train.samples/train.batch_size;
#valid_steps = valid.samples/valid.batch_size;

#with tf.device('/gpu:0'):
#            model.fit_generator(train, steps_per_epoch = train_steps, epochs = 1, 
#                                validation_data = valid, 
#                                validation_steps = valid_steps,                 
#                               )
#            for i in range(4):
#                pseudo = PseudoLabeling(model, train, valid, path)
#                print ("Predicting validation labels")
#                pseudo.predict_validation_labels()
#                print("Done predicting validation labels")
#                model.fit_generator(pseudo, steps_per_epoch = train_steps, epochs = 1,
#                                    validation_data = valid,
#                                    validation_steps = valid_steps,
#                                   )
#
#            model.save_weights(path + 'weights_exp10.h5')

In [47]:
import math
import os
import numpy as np
from keras.preprocessing.image import Iterator
from keras.preprocessing.image import img_to_array, load_img
import keras.backend as K
data_path = '/work/04381/ymarathe/maverick/yearbook/'

class RegressDataGen:
    def __init__(self, directory, map_file):
        self.directory = directory
        self.map_file = map_file
        self.filenames = []
        self.map = {}
        self.populate_filenames()
        self.populate_mapping()
        self.regressIter = None
        self.steps = 0
        
    def _recursive_list(self, subpath):
        return sorted(
            os.walk(subpath, followlinks=False), key=lambda tpl: tpl[0])
    
    def populate_mapping(self):
        f = open(self.map_file, 'r')

        for line in f:
            line = line.rstrip()
            image, year = line.split("\t")
            self.map[image] = year
            
    def populate_filenames(self):
        base_dir = self.directory
        for root, _, files in self._recursive_list(base_dir):
            for fname in files:
                if fname.lower().endswith('.' + 'png'):
                    self.filenames.append(os.path.relpath(os.path.join(root, fname), base_dir))
                    
    def flow_from_directory(self, batch_size = 32, shuffle = True, seed = 42, target_size = (171, 186)):
        self.regressIter = Iterator(len(self.filenames), batch_size = batch_size, shuffle = shuffle, seed = seed)
        self.steps = math.ceil(len(self.filenames)/batch_size)
        return self
    
    def next(self, *args, **kwargs):
        target_size = (171, 186, 3)
           
        idx_array, cur_idx, bs = next(self.regressIter.index_generator)
        
        batch_x = np.zeros(tuple([len(idx_array)] + list(target_size)), dtype=K.floatx())
        
        batch_y = np.zeros(tuple([len(idx_array)]), dtype=K.floatx())
        
        for i, j in enumerate(idx_array):
            fname = self.filenames[j]
            img = load_img(
                  os.path.join(self.directory, fname),
                  grayscale = True,
                  target_size= target_size)
            x = np.array(img_to_array(img, data_format='channels_last'))
            batch_x[i] = x
            batch_y[i] = self.map[fname]
        
        return (batch_x, batch_y)

In [48]:
train = RegressDataGen(data_path + 'train',
                             data_path + 'yearbook_train.txt')
valid = RegressDataGen(data_path + 'valid',
                             data_path + 'yearbook_valid.txt')

train = train.flow_from_directory()
valid = valid.flow_from_directory()

In [58]:
mean_value = 0
for key in freq:
    mean_value += freq[key] * float(key)
    
model = vgg19_conv_layers_sequential()
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(4096, kernel_initializer='glorot_normal', bias_initializer=keras.initializers.Ones()))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1, activation='relu', bias_initializer=keras.initializers.Constant(mean_value)))

for layer in model.layers:
    if type(layer) != Dense:
        layer.trainable = False

model.compile(Adam(lr=1e-2), loss='mse', metrics=['accuracy', 'mae'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_81 (ZeroPaddi (None, 173, 188, 3)       0         
_________________________________________________________________
conv2d_81 (Conv2D)           (None, 171, 186, 64)      1792      
_________________________________________________________________
zero_padding2d_82 (ZeroPaddi (None, 173, 188, 64)      0         
_________________________________________________________________
conv2d_82 (Conv2D)           (None, 171, 186, 64)      36928     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 85, 93, 64)        0         
_________________________________________________________________
zero_padding2d_83 (ZeroPaddi (None, 87, 95, 64)        0         
_________________________________________________________________
conv2d_83 (Conv2D)           (None, 85, 93, 128)       73856     
__________

with tf.device('/gpu:0'):
           model.fit_generator(train, steps_per_epoch = train.steps, epochs = 1,                                
                                validation_data = valid, 
                                validation_steps = valid.steps, 
                               )
           model.save_weights(path + 'weights_exp11.h5')

**Experiment 11**
- Model as a regression problem
- Metrics = accuracy, mae
- One dense layer (4096), BatchNorm, second dense layer (1)
- No dropout
- lr = 1e-3
- Slow convergence
Epoch 1/1

713/713 [==============================] - 429s - loss: 1318.5696 - acc: 4.3829e-05 - mean_absolute_error: 1318.5696 - val_loss: 627.1355 - val_acc: 4.0064e-04 - val_mean_absolute_error: 627.1355

- lr = 1e-2

Total params: 72,528,961

Trainable params: 52,436,993

Non-trainable params: 20,091,968

Epoch 1/1

713/713 [==============================] - 428s - loss: 348.7608 - acc: 0.0013 - mean_absolute_error: 348.7608 - val_loss: 302.1588 - val_acc: 0.0010 - val_mean_absolute_error: 302.1588

- lr = 1e-2
- bias_initializer = mean of values

Epoch 1/1

713/713 [==============================] - 428s - loss: 11.1228 - acc: 0.0319 - mean_absolute_error: 11.1228 - val_loss: 10.2767 - val_acc: 0.0351 - val_mean_absolute_error: 10.2767


**Experiment 12**
- More number of epochs

Total params: 72,528,961

Trainable params: 52,436,993

Non-trainable params: 20,091,968

Epoch 1/4

713/713 [==============================] - 428s - loss: 11.1498 - acc: 0.0313 - mean_absolute_error: 11.1498 - val_loss: 9.9395 - val_acc: 0.0453 - val_mean_absolute_error: 9.9395

Epoch 2/4

713/713 [==============================] - 472s - loss: 9.5142 - acc: 0.0376 - mean_absolute_error: 9.5142 - val_loss: 11.3703 - val_acc: 0.0340 - val_mean_absolute_error: 11.3703

Epoch 3/4

713/713 [==============================] - 423s - loss: 8.9035 - acc: 0.0401 - mean_absolute_error: 8.9035 - val_loss: 12.3618 - val_acc: 0.0336 - val_mean_absolute_error: 12.3618

Epoch 4/4

713/713 [==============================] - 423s - loss: 8.5783 - acc: 0.0413 - mean_absolute_error: 8.5783 - val_loss: 12.2621 - val_acc: 0.0323 - val_mean_absolute_error: 12.2621


In [ ]:
with tf.device('/gpu:0'):
           model.fit_generator(train, steps_per_epoch = train.steps, epochs = 4,                                
                                validation_data = valid, 
                                validation_steps = valid.steps
                               )
           model.save_weights(path + 'weights_exp12.h5')